In [12]:
#coding:utf-8

import keras
import gensim
from keras.utils import plot_model

VECTOR_DIR = 'baike.vectors.bin'

MAX_SEQUENCE_LENGTH = 100
EMBEDDING_DIM = 200
VALIDATION_SPLIT = 0.16
TEST_SPLIT = 0.2



train_texts = open('train_x.txt',encoding='utf-8').read().split('\n')
train_labels = open('train_y.txt').read().split('\n')
test_texts = open('test_x.txt',encoding='utf-8').read().split('\n')
test_labels = open('test_y.txt').read().split('\n')
all_texts = train_texts + test_texts
all_labels = train_labels + test_labels


from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
import numpy as np

tokenizer = Tokenizer()
tokenizer.fit_on_texts(all_texts)
sequences = tokenizer.texts_to_sequences(all_texts)
word_index = tokenizer.word_index

data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
labels = to_categorical(np.asarray(all_labels))

p1 = int(len(data)*(1-VALIDATION_SPLIT-TEST_SPLIT))
p2 = int(len(data)*(1-TEST_SPLIT))
x_train = data[:p1]
y_train = labels[:p1]
x_val = data[p1:p2]
y_val = labels[p1:p2]
x_test = data[p2:]
y_test = labels[p2:]

w2v_model = gensim.models.KeyedVectors.load_word2vec_format(VECTOR_DIR, binary=True)
embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))
not_in_model = 0
in_model = 0
for word, i in word_index.items(): 
    if str(word) in w2v_model:
        in_model += 1
        embedding_matrix[i] = np.asarray(w2v_model[str(word)], dtype='float32')
    else:
        not_in_model += 1
print (str(not_in_model)+' words not in w2v model')
from keras.layers import Embedding
embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)



from keras.layers import Dense, Input, Flatten, Dropout
from keras.layers import Conv1D, MaxPooling1D, Embedding, GlobalMaxPooling1D
from keras.models import Sequential

model = Sequential()
model.add(embedding_layer)
model.add(Dropout(0.2))
model.add(Conv1D(250, 3, padding='valid', activation='relu', strides=1))
model.add(MaxPooling1D(3))
model.add(Flatten())
model.add(Dense(EMBEDDING_DIM, activation='relu'))
model.add(Dense(labels.shape[1], activation='softmax'))
model.summary()
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])
print (model.metrics_names)
model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=2, batch_size=128)
model.save('word_vector_cnn.h5')
print (model.evaluate(x_test, y_test))

(1) load texts...
(2) doc to var...
Found 65604 unique tokens.
Shape of data tensor: (21924, 100)
Shape of label tensor: (21924, 12)
3) split data set...
train docs: 14031
val docs: 3508
test docs: 4385
(4) load word2vec as embedding...
13822 words not in w2v model
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 100, 200)          13121000  
_________________________________________________________________
dropout_7 (Dropout)          (None, 100, 200)          0         
_________________________________________________________________
conv1d_7 (Conv1D)            (None, 98, 250)           150250    
_________________________________________________________________
max_pooling1d_7 (MaxPooling1 (None, 32, 250)           0         
_________________________________________________________________
flatten_7 (Flatten)          (None, 8000)              0         
________